In [ ]:
# ! pip install xgboost
# !pip install imblearn
# !pip install imbalanced-learn
# !pip install -U scikit-learn

In [45]:
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics import confusion_matrix

#for model-building
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfTransformer

from imblearn.over_sampling  import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import SMOTE


from imblearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV

import pickle

In [7]:
news_ids = pd.read_csv(r'../data/news_outlets-accounts.csv')
news_ids.shape

(9636, 3)

In [8]:
news_ids.head()

,Token,Uid,Link
0,LouisvilleMag,11263532,https://twitter.com/LouisvilleMag
1,WindyCityTimes,30284449,https://twitter.com/WindyCityTimes
2,BlkMtnNews,3053471913,https://twitter.com/BlkMtnNews
3,NewYorker,14677919,https://twitter.com/NewYorker
4,KyKernel,41463945,https://twitter.com/KyKernel


In [10]:
input_path = r'../components/artifacts/preprocess_data.csv'
preprocess_data = (
            pd.read_csv(input_path, sep = "\t")
            .drop_duplicates()
        )
preprocess_data["Full Text"] =preprocess_data["Full Text"].fillna("")
preprocess_data.shape

(7125059, 21)

In [11]:
preprocess_data.head()

,Unnamed: 0,Date,Page Type,Account Type,Author,Full Name,Full Text,Gender,Hashtags,Impact,...,Thread Entry Type,Thread Author,Twitter Followers,Twitter Following,Twitter Tweets,Twitter Reply Count,Twitter Verified,Twitter Retweets,Reach (new),Region
0,0,2017-12-06 23:59:59,twitter,NaN,William82861606,William82861606 (William),funder conyers resigned franken resigning tomo...,male,"#trumpsexprobe, #metoo",0.0,...,share,funder,282.0,415.0,17340.0,0.0,False,0.0,0.0,Arizona
1,1,2017-12-06 23:59:58,twitter,NaN,Artis_Elemental,Artis_Elemental (ART...Is),alyssa_milano list sexual misconduct allegatio...,male,#metoo,24.5,...,share,Alyssa_Milano,5578.0,5769.0,363570.0,0.0,False,0.0,3311.0,NaN
2,2,2017-12-06 23:59:57,twitter,individual,lanah03,lanah03 (Laura Hopkins),deepinthehills sobbing silent longer,female,#metoo,5.4,...,share,DeepInTheHills,2713.0,4978.0,46914.0,0.0,False,0.0,1734.0,New York
3,3,2017-12-06 23:59:55,twitter,individual,lidskawasaki,lidskawasaki (dearlordbabyjesus),alyssa_milano time asking franken resign senat...,unknown,"#timepersonoftheyear, #metoo",0.0,...,share,Alyssa_Milano,330.0,273.0,10901.0,0.0,False,0.0,0.0,NaN
4,4,2017-12-06 23:59:54,twitter,NaN,IAmMeliLopez,IAmMeliLopez (Melissa Lopez),nycchr public hearing begun stories deserve he...,female,"#sexualharassment, #metoo",0.0,...,share,NYCCHR,13.0,96.0,11594.0,0.0,False,0.0,0.0,New York


In [12]:
preprocess_data.columns

Index(['Unnamed: 0', 'Date', 'Page Type', 'Account Type', 'Author',
       'Full Name', 'Full Text', 'Gender', 'Hashtags', 'Impact', 'Impressions',
       'Thread Entry Type', 'Thread Author', 'Twitter Followers',
       'Twitter Following', 'Twitter Tweets', 'Twitter Reply Count',
       'Twitter Verified', 'Twitter Retweets', 'Reach (new)', 'Region'],
      dtype='object')

In [22]:
news_id_list = news_ids['Token'].tolist()
# preprocess_data['news_authors'] = preprocess_data['Author'].apply(lambda x: x in news_id_list)

# preprocess_data[['Author','news_authors']].head()

In [14]:
# preprocess_data[preprocess_data['news_authors']][['Author','news_authors','Full Text']].head()

In [15]:
# preprocess_data[preprocess_data['news_authors']][['Author','news_authors']].shape

In [16]:
# def news_identification_heuristic(author,tweet):
#     # print(author,tweet)
#     auth_heuristics = 'news' in str(author).lower() or 'times' in str(author).lower()
#     tweet_heuristics = 'breaking news' in str(tweet).lower()
#     return auth_heuristics or tweet_heuristics

In [17]:
# preprocess_data['news_authors_simp_heu'] = preprocess_data[['Author','Full Text']].\
#     apply(lambda x: news_identification_heuristic(x[0],x[1]),axis=1)
# # preprocess_data[['Author','news_authors_simp_heu']].head()
# preprocess_data[preprocess_data['news_authors_simp_heu']][['Author','news_authors_simp_heu','Full Text']].head()

In [19]:
# preprocess_data[preprocess_data['news_authors_simp_heu']][['Author','news_authors_simp_heu']].shape


In [18]:
# preprocess_data[preprocess_data['news_authors_simp_heu']][['Author','news_authors_simp_heu','news_authors','Full Text']].head(5)

In [20]:
# news_id_list = [x.lower() for x in news_id_list]
def news_iden_imp_heu(author):
    # print(author,tweet)
    auth_heuristics = 'news' in str(author).lower() or author in news_id_list
    return 1 if auth_heuristics else 0

In [23]:
preprocess_data["Full Text"] =preprocess_data["Full Text"].fillna("")
author_corpus = preprocess_data.groupby("Author")["Full Text"].apply(" ".join).reset_index()
author_corpus ['news_label'] = author_corpus['Author'].apply(news_iden_imp_heu)
author_corpus[author_corpus['news_label']==1]

,Author,Full Text,news_label
804,1003ThePeakABQ,baby cold outside cut cleveland radio station ...,1
809,1007WRDU,gene simmons worries collateral damage movemen...,1
861,100dailynews1,amazon defends dropping woody allen film deal ...,1
892,1011_News,bill cosby convicted drugging molesting woman ...,1
895,1011wjrr,gene simmons worries collateral damage movement,1
...,...,...,...
1364966,ysbnews,hotlinejosh tocradio apparently affected soft ...,1
1365935,z1077,gabrielle union opens twitter campaign gma vid...,1
1367890,zestynews_paleo,new original sin finally absolves eve feminist...,1
1368593,znewsafrica,chimamanda says ta nehisi coates ibo hillary r...,1


In [24]:
author_corpus

,Author,Full Text,news_label
0,00000365,malmet harvey weinstein found guilty landmark ...,0
1,0000Baeza,ravebeautybar want hands training canadaelecti...,0
2,0000timmy,funder women accused trump sexual assault tell...,0
3,0001nvywrvet,doublestandards liberals walkaway resist abuser,0
4,0002RG,mari sarutobi abc time nbcnews foxnews bbc bbc...,0
...,...,...,...
1369765,zzzzaaaacccchhh,janetmaslin need ideal poster person believe w...,0
1369766,zzzzfunk,alyssa_milano dont let performance fool senato...,0
1369767,zzzzxxcbv,funder rep speier th member congress trump res...,0
1369768,zzzzzac12,queenbpip justice johnny depp thread johnnydepp,0


In [25]:
vec = TfidfVectorizer(max_features = 20000)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
        author_corpus['Full Text'], 
        author_corpus['news_label'],
        random_state = 123,
        train_size = 0.75,
        
    )

In [40]:
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

In [41]:
model = MultinomialNB()
model.fit(X_train_vec, y_train)

MultinomialNB()

In [42]:
train_preds = model.predict(X_train_vec)
test_preds = model.predict(X_test_vec)
train_preds

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
print(classification_report(y_test, test_preds))
print(confusion_matrix(y_test, test_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    340773
           1       0.41      0.03      0.06      1670

    accuracy                           1.00    342443
   macro avg       0.70      0.52      0.53    342443
weighted avg       0.99      1.00      0.99    342443

[[340691     82]
 [  1614     56]]


In [ ]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vec, y_train)

In [ ]:
train_preds = lr_tfidf.predict(X_train_vec)
test_preds = lr_tfidf.predict(X_test_vec)
# train_preds

In [ ]:
print(classification_report(y_test, test_preds))
print(confusion_matrix(y_test, test_preds))

In [ ]:
cv = CountVectorizer(max_features=10000, encoding="utf-8",  
      ngram_range = (1,3),  
      token_pattern = "[A-Za-z_][A-Za-z\d_]*")
X_train_vec = cv.fit_transform(X_train)
X_test_vec = cv.transform(X_test)

In [ ]:
# model = XGBClassifier()
# model.fit(X_train, y_train)

# # make predictions for test data

# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]

In [ ]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)
parameters = {
    'grow_policy' : [0,1],
    'max_depth': range (2, 20, 1),
    'n_estimators': range(50, 250, 10),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'balanced_accuracy',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

# xgb_model = grid_search.fit(X_train_vec, y_train)

In [ ]:
# xgb_model.param_grid

/home/sridhar.p/.local/lib/python3.8/site-packages/imblearn/utils/_validation.py:586: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [36]:
nb_pipe2  = Pipeline([('vect',    CountVectorizer()),
                     ('tfidf',   TfidfTransformer()),
                     ('sampler', RandomOverSampler('minority',random_state=42)),
                     ('model',   MultinomialNB())])
model = nb_pipe2.fit(X_train, y_train)

pred  = model.predict(X_test)

print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

/home/sridhar.p/.local/lib/python3.8/site-packages/imblearn/utils/_validation.py:586: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      0.89      0.94    340773
           1       0.03      0.66      0.05      1670

    accuracy                           0.88    342443
   macro avg       0.51      0.77      0.50    342443
weighted avg       0.99      0.88      0.93    342443

[[301707  39066]
 [   561   1109]]


In [37]:
nb_pipe3  = Pipeline([('vect',    CountVectorizer()),
                     ('tfidf',   TfidfTransformer()),
                     ('sampler', SMOTE('minority',random_state=42)),
                     ('model',   MultinomialNB())])

model = nb_pipe3.fit(X_train, y_train)

pred  = model.predict(X_test)

print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

/home/sridhar.p/.local/lib/python3.8/site-packages/imblearn/utils/_validation.py:586: FutureWarning: Pass sampling_strategy=minority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      0.88      0.94    340773
           1       0.03      0.66      0.05      1670

    accuracy                           0.88    342443
   macro avg       0.51      0.77      0.49    342443
weighted avg       0.99      0.88      0.93    342443

[[300088  40685]
 [   562   1108]]


In [48]:
nb_pipe3  = Pipeline([('vect',    CountVectorizer()),
                     ('tfidf',   TfidfTransformer()),
                     ('sampler', RandomUnderSampler('majority',random_state=42)),
                     ('model',   MultinomialNB())])

model = nb_pipe3.fit(X_train, y_train)

pred  = model.predict(X_test)

print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

/home/sridhar.p/.local/lib/python3.8/site-packages/imblearn/utils/_validation.py:586: FutureWarning: Pass sampling_strategy=majority as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


              precision    recall  f1-score   support

           0       1.00      0.73      0.84    340773
           1       0.01      0.83      0.03      1670

    accuracy                           0.73    342443
   macro avg       0.51      0.78      0.44    342443
weighted avg       0.99      0.73      0.84    342443

[[247870  92903]
 [   286   1384]]


In [46]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [47]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

ValueError: could not convert string to float: 'tater daughter requesting attend high school rapist denied isn considered extreme hardship friggen kidding wtf wrong county shouldn appeal jcps_nc wral abc cnn witn'